# Marketing Growth Analysis - Mini Conversion Funnel & Performance Analysis

## CONVERSION FUNNEL ANALYSIS
* Its about understanding and Optimizing each step of the User Experience on their Journey towards purchase of our product 
* Use Case 1- Identifying the most common Paths customers take before purchasing our product 
* Use Case 2- Identifying how many of our users continue on to each next step in conversion flow & how many abandon at each step
* Use Case 3- Optimizing critical pain points where users are abandoning so that you can convert more users & sell more products 
    * Example 95 users  visited home page, 65 from among those 95 visited products page, 45 from among those 65 visited  to add to cart page for final purchase
    * So 65/95 gives us the Click through rate CTR for Products Page and so on ...

### We want to build a mini conversion funnel from a Landing Page ('lander-1') to Shopping Cart ('/cart')
### We want to know how many people reach each step in their Customer Journey, and drop off  rates
### We are focussing at customer who are looking at product "Mr Fuzzy only" (i.e "/the-original-mr-fuzzy")


In [7]:
# importing Libraries
import os
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error

In [ ]:
!pip install numpy, pandas

In [8]:
df = pd.read_csv('website_pageviews.csv',header=0)
df.head()

,website_pageview_id,created_at,website_session_id,pageview_url
0,1,2012-03-19 08:04:16,1,/home
1,2,2012-03-19 08:16:49,2,/home
2,3,2012-03-19 08:26:55,3,/home
3,4,2012-03-19 08:37:33,4,/home
4,5,2012-03-19 09:00:55,5,/home


## Create a SQL Lite In Memory DB Connection

In [10]:

import sqlite3
from sqlite3 import Error
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

### Reading CSV Files into Data Frames and then writing into SQL Lite Tables (In-Memory)
### Reading from SQL Lite Tables, Performing necessary aggregations and displaying 
### Displaying few rows from session table to get an Idea of the table structure

In [13]:
conn=create_connection(':memory:')
#website_sessions_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("website_sessions.csv")
website_pageviews_df = pd.read_csv('website_pageviews.csv',header=0)
website_pageviews_df.head()
website_sessions_df = pd.read_csv('website_sessions.csv',header=0)
website_sessions_df.head()
orders_df = pd.read_csv('orders_maven.csv',header=0)
#website_sessions_df = pd.read_csv('website_pageviews.csv')
cursor = conn.cursor()
tbl_name1= "website_pageviews"
cursor.execute("drop table if exists %s;"%(tbl_name1))
#print('Table {0} was created successfully'.format(tbl_name1))
website_pageviews_df.to_sql(tbl_name1,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name1))
tbl_name2= "website_sessions"
cursor.execute("drop table if exists %s;"%(tbl_name2))
#print('Table {0} was created successfully'.format(tbl_name2))
website_sessions_df.to_sql(tbl_name2,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name2))
# creating a third table [orders] in memory
tbl_name3= "orders"
cursor.execute("drop table if exists %s;"%(tbl_name3))
#print('Table {0} was created successfully'.format(tbl_name2))
orders_df.to_sql(tbl_name3,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name3))

conn.commit()
#cursor=conn.cursor()
sql="select * from %s limit 10"%tbl_name1
sql_df1 = pd.read_sql('select * from website_sessions limit 3',con=conn)
sql_df2 = pd.read_sql('select * from website_pageviews limit 3',con=conn)
sql_df1.head(3)
#sql_df2.head(3)

File website_pageviews copied to db
File website_sessions copied to db
File orders copied to db


,website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
0,1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
1,2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
2,3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com


## CONVERSION FUNNEL ANALYSIS
* Its about understanding and Optimizing each step of the User Experience on their Journbey towards purchase of our product 
* Use Case 1- Identifying the most common Paths customers take before purchasing our product 
* Use Case 2- Identifying how many of our users continue on to each next step in conversion flow & how mnay abandon at each step
* Use Case 3-  Optimizing critical pain points where users are abandoning so that you can convert more users & sell more products 
    * Example 95 users  visited home page, 65 from among those 95 visited products page, 45 from among those 65 visited  to add to cart page for final purchase
    * So 65/95 gives us the Click through rate CTR for Products Page and so on ...

In [14]:

session_level_made_it_flag_demo = pd.read_sql("""
select website_session_id, 
max(products_page) products_made_it,
max(mrfuzzy_page) mrfuzzy_made_it,
max(cart_page) cart_page_made_it
from 
(
select website_sessions.website_session_id, 
website_pageviews.pageview_url, 
website_pageviews.created_at pageview_created_at,
case when website_pageviews.pageview_url= '/products' then 1 else 0 end as products_page,
case when website_pageviews.pageview_url= '/the-original-mr-fuzzy' then 1 else 0 end as mrfuzzy_page,
case when website_pageviews.pageview_url= '/cart' then 1 else 0 end as cart_page
from website_sessions left join website_pageviews
on website_sessions.website_session_id = website_pageviews.website_session_id
where 
	 --website_pageviews.pageview_url = (select min(pageview_url) from website_pageviews group by website_session_id)
	--website_sessions.created_at>='2012-03-19 08:26:55' and 
website_pageviews.pageview_url in ('/the-original-mr-fuzzy','/lander-2','/products','/cart') 
order by website_sessions.website_session_id
)x	
group by website_session_id
order by website_session_id
"""
,con=conn)
tbl_name3= "session_level_made_it_flag_demo"
session_level_made_it_flag_demo.to_sql(tbl_name3,conn,if_exists='replace', index=False)

In [15]:
pd.read_sql("""
select count(distinct website_session_id) as sessions,
round(count(distinct case when products_made_it=1 then website_session_id else null end )*1.0/
count(distinct website_session_id),3) as lander_click_through_rate , 
round(count(distinct case when mrfuzzy_made_it =1 then website_session_id else null end )*1.0/
count(distinct website_session_id),3) as products_through_rate,
ROUND(count(distinct case when cart_page_made_it =1 then website_session_id else null end )*1.0/
count(distinct website_session_id),3) as mrfuzzy_click_through_rate
from session_level_made_it_flag_demo
"""
,con=conn).head()

,sessions,lander_click_through_rate,products_through_rate,mrfuzzy_click_through_rate
0,320480,0.815,0.507,0.296


# Great Job!

<h2> Marketing Campaign Performance Analysis 
</h2>
<h3> ABC Company is into business for past 8 months and has ran various Digitial Marketing Campaigns to boost the sales. Marketing team has questions about the peformance of the campaign and would like to analyse the customer journey characteristics and related opportinities/problems
</h3>
<h4> Q # 01 - Marketing Team believes that Google Search (gsearch)  seems to be the biggest driver of the Business, Kindly pull out the monthly trends for search sessions and orders so that we can showcase the growth there

In [16]:
pd.read_sql("""
select strftime('%Y',website_sessions.created_at) yr,
strftime('%m',website_sessions.created_at) mo,
count(distinct website_sessions.website_session_id) sessions, 
 count(distinct orders.order_id) orders,
 round(count(distinct orders.order_id)*1.0/count(distinct website_sessions.website_session_id)*100,2) as conversion_rate
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where utm_source='gsearch' 
and  website_sessions.created_at < '2012-11-27'
group by strftime('%Y',website_sessions.created_at),strftime('%m',website_sessions.created_at)
"""
,con=conn).head(10)

,yr,mo,sessions,orders,conversion_rate
0,2012,03,1860,60,3.23
1,2012,04,3574,92,2.57
2,2012,05,3410,97,2.84
3,2012,06,3578,121,3.38
4,2012,07,3811,145,3.80
5,2012,08,4877,184,3.77
6,2012,09,4491,188,4.19
7,2012,10,5534,234,4.23
8,2012,11,8889,373,4.20


<h3> RESULT - We can see that the Orders started growing in  number starting June Month

<h3> Q # 02
<h4> Now Marketing team would like to split the 'brand' and 'non-brand' campaign separately for Google Search(gsearch)
<h4> Marketing team would like to know if campaign named 'brand' is pickign up or not 
<h4> Brand (brand) here means that the customer is directly using our name in Google Search Engine

In [17]:
pd.read_sql("""
select strftime('%Y',website_sessions.created_at) yr,
	strftime('%m',website_sessions.created_at) yr_month,--utm_campaign,
	count(distinct  case when  utm_campaign = 'nonbrand' then website_sessions.website_session_id else null end ) nonbrand_sessions, 
	count(distinct case when  utm_campaign = 'nonbrand' then orders.order_id else null end ) nonbrand_orders,
	count(distinct  case when  utm_campaign = 'brand' then website_sessions.website_session_id else null end ) brand_sessions, 
 	count(distinct case when  utm_campaign = 'brand' then orders.order_id else null end ) brand_orders
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where utm_source='gsearch' and utm_campaign in ('nonbrand', 'brand')
and  website_sessions.created_at < '2012-11-27' ---- 19-03-2012
group by strftime('%Y',website_sessions.created_at) ,
strftime('%m',website_sessions.created_at)
"""
,con=conn).head(10)

,yr,yr_month,nonbrand_sessions,nonbrand_orders,brand_sessions,brand_orders
0,2012,03,1852,60,8,0
1,2012,04,3509,86,65,6
2,2012,05,3295,91,115,6
3,2012,06,3439,114,139,7
4,2012,07,3660,136,151,9
5,2012,08,4673,174,204,10
6,2012,09,4227,172,264,16
7,2012,10,5197,219,337,15
8,2012,11,8506,356,383,17


</h3> Result, We can see 'brand' is gradually picking up  on a month-to-month basis 

<h2> Q # 03
<h4> Now we would like to dive into non-brand and pull the monthly sessions and orders split by device type i.e mobile, desktop 

In [19]:
pd.read_sql("""
select strftime('%Y',website_sessions.created_at) yr,
	strftime('%m',website_sessions.created_at)yr_mo,
	Count(distinct  case when device_type = 'desktop' then website_sessions.website_session_id else null end ) desktop_sessions, 
	count(distinct case when  device_type = 'desktop' then orders.order_id else null end ) desktop_orders,
	count(distinct  case when device_type = 'mobile' then website_sessions.website_session_id else null end ) mobile_sessions, 
 	count(distinct case when  device_type = 'mobile' then orders.order_id else null end ) mobile_orders
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where utm_source='gsearch' and utm_campaign = 'nonbrand'
and  website_sessions.created_at < '2012-11-27' ---- 19-03-2012
group by strftime('%Y',website_sessions.created_at),
strftime('%m',website_sessions.created_at)
""" ,con=conn).head(10)

,yr,yr_mo,desktop_sessions,desktop_orders,mobile_sessions,mobile_orders
0,2012,03,1128,50,724,10
1,2012,04,2139,75,1370,11
2,2012,05,2276,83,1019,8
3,2012,06,2673,106,766,8
4,2012,07,2774,122,886,14
5,2012,08,3515,165,1158,9
6,2012,09,3171,155,1056,17
7,2012,10,3934,201,1263,18
8,2012,11,6457,323,2049,33


<h4> It can be seen that most of the orders are coming from Desktop and we witness overall growth from 3rd Month through 11 Month 

<h3> Q # 04
<h4> Although Google Search (gsearch ) is working fine, but one of the Marketing team member is concerned about the large %age from
<h4> Google Search, Lets pull a monthly trends for Google Search along with monthly trends for each of the other channels 

In [20]:
pd.read_sql("""
select 
	strftime('%Y',website_sessions.created_at) yr,
	strftime('%m',website_sessions.created_at) yr_month,
	count(distinct case when utm_source='gsearch' then  website_sessions.website_session_id else null end) 
	google_paid_sessions, 
	count(distinct case when utm_source='bsearch' then  website_sessions.website_session_id else null end) 
	bing_paid_sessions, 
	count(distinct case when utm_source is null and http_referer is not null then  website_sessions.website_session_id else null end) 
	organic_search_sessions, 
	count(distinct case when utm_source is null and http_referer is null then  website_sessions.website_session_id else null end) 
	direct_type_in_sessions
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where -- utm_source='gsearch' --and utm_campaign = 'nonbrand' and 
website_sessions.created_at < '2012-11-27' ---- 19-03-2012
group by strftime('%Y',website_sessions.created_at), strftime('%m',website_sessions.created_at)
""" ,con=conn).head(10)

,yr,yr_month,google_paid_sessions,bing_paid_sessions,organic_search_sessions,direct_type_in_sessions
0,2012,03,1860,2,8,9
1,2012,04,3574,11,78,71
2,2012,05,3410,25,150,151
3,2012,06,3578,25,190,170
4,2012,07,3811,44,207,187
5,2012,08,4877,705,265,250
6,2012,09,4491,1439,331,285
7,2012,10,5534,1781,428,440
8,2012,11,8889,2840,536,485


<h4> We can see gsearch is still the largest contributor followed by blank/un-known channel 

<h3> Q # 05
